In [89]:
from PyCAR.PyCIT.FT import MeshFlux,MeshPower
import numpy as np
from PyCAR.PyCIT.FT import GroupFlux
from PyCAR.PyCIT.lector_mest_posta import get_reactivity_pro as grp
import re
import os
import matplotlib.pyplot as plt

In [90]:
BetaDir='D:\\Cube\\'
#FILENAME = 'NoobCube2gS.cii'
FILENAME = 'NoobCube1gS.cii'
ROOTFILE = FILENAME.split('.')[0]
ROOTFILE

'NoobCube1gS'

In [91]:
Nodosx=44
Nodosy=22
Nodosz=32
#Ngrupos=2
Ngrupos=1
#Flux = MeshFlux(BetaDir+'NoobCube2g.cdb.meshflux',Nodosx,Nodosy,Nodosz,Ngrupos)
Flux = MeshFlux(BetaDir+ROOTFILE[:-1]+'.cdb.meshflux',Nodosx,Nodosy,Nodosz,Ngrupos)
Flux.shape

(1, 44, 22, 32, 1)

In [92]:
BetaDir+ROOTFILE[:-1]+'.cdb.meshflux'

'D:\\Cube\\NoobCube1g.cdb.meshflux'

TODOS ESTOS VALORES FUERON SACADOS DE CONDOR, DEL EC 1.8% CON SEPARADOR

In [93]:
NPRC = 1
NOGBIB = Flux.shape[-1]
#NuFis=[2.0150E-02]

lmks=[0.0124,0.0306,0.1135,0.3071,1.1905,3.1748]
Betas=[0.0002170,0.0014979,0.0013778,0.0028296,0.0009288,0.0003314,0.0071826]
BetT=Betas[-1]

#valor obtenido de NoobCube2g.cii
#keff = 1.0005462
keff = 1/(1-grp(ROOTFILE[:-1]+'.plt')[-1]/100000)

#chi_g=[0.999827,0.000173]
chi_g=[0.9999]

#NuFis=[2.8132E-03,3.6287E-02]
#NuFis=[2.81322002E-03,3.62874009E-02]
NuFis=[2.00051703E-02]
Abs=[2.02931999E-03,3.27022001E-02]

NuFis

[0.0200051703]

In [94]:
sum(Betas[:-1])

0.0071825000000000005

In [95]:
def Initialize(iterable):
    IterM={}#matriz de betas
    for state in range(Flux.shape[0]):
        IterM[state]={}
        for nx in range(Flux.shape[1]):
            IterM[state][nx] = {}
            for ny in range(Flux.shape[2]):
                IterM[state][nx][ny] = {}
                for nz in range(Flux.shape[3]):
                    IterM[state][nx][ny][nz]= iterable
    return IterM

if NPRC == 6:
    BetaM = Initialize(Betas)#6 grupos de precursores
else:
    BetaM = Initialize([Betas[-1]])#1 grupo de precursores

In [96]:
if NPRC == 6:
    LmkM = Initialize(lmks)
else:
    LmkM = Initialize([Betas[-1]/sum([Betas[k]/lmks[k] for k in range(6)])])

In [97]:
NuFisM = Initialize({group:NuFis[group] for group in range(Flux.shape[-1])})

# LA FUENTE SE DA EN NEUTRONES POR SEGUNDO,
# NO EN NEUTRONES/CM3

SE SUBESTIMA LA CANTIDAD DE PRODUCCIONES

GFlux=GroupFlux(BetaDir+'NoobCube2g.cdb.flux')
GFlux.shape

In [98]:
Vmesh = (6*6*6)
Vcell = 6*Vmesh

In [99]:
Vmesh

216

FUENTE FOR DUMMIES, confirmado que devuelve 1 MW

kFlux = MeshFlux(BetaDir+'NoobCube2g.cdb.meshflux',Nodosx,Nodosy,Nodosz,Ngrupos)
kpower = MeshPower('NoobCube2g.cdb.meshpower',Nodosx,Nodosy,Nodosz)
kpower1 = MeshPower('NoobCube1g.cdb.meshpower',Nodosx,Nodosy,Nodosz)

FissRate = NuFis[0]*kFlux[:,:,:,:,0]+NuFis[1]*kFlux[:,:,:,:,1]
AbssRate = Abs[0]*kFlux[:,:,:,:,0]+Abs[1]*kFlux[:,:,:,:,1]

Fissions = FissRate*Vmesh
Abssorts = AbssRate*Vmesh

Precs = BetT*Fissions

NewP = Fissions.sum()
NewA = Abssorts.sum()

'{:.5e}'.format(NewP)

'{:.5e}'.format(NewA)

EFis = [3.44858013E-14,4.83173993E-13]
FissPower = EFis[0]*kFlux[...,0]+EFis[1]*kFlux[...,1]
plt.plot(range(32),FissPower[0,22,11,:],range(32),kpower1[0,22,11,:]/100,'o')
plt.show()

'{:.5e}'.format((kpower1*Vmesh).sum())

$$ Precs_{i} = \beta_{T}  \sum_g V_{mesh,i}   \frac{\nu\Sigma_{f,g}}{k_{eff}} \phi_g, \beta_{T} = \sum_k \beta_k $$
$$ C_{i,k,0} = \frac{\beta_{k}}{\lambda_{k}} \sum_g V_{mesh,i} \frac{\nu \Sigma_{f,g}}{k_{eff}} \phi_g$$

In [100]:
from functools import reduce
from operator import add

def EForm(*S):
    return reduce(add,['{:.8e} '.format(_s) for _s in S])

C0 = {}
for state in range(Flux.shape[0]):
    C0[state]={}
    for nx in range(Flux.shape[1]):
        C0[state][nx] = {}
        for ny in range(Flux.shape[2]):
            C0[state][nx][ny] = {}
            for nz in range(Flux.shape[3]):
                
                FluxL = [Flux[state,nx,ny,nz,group] for group in range(Flux.shape[-1])]
                NuFisL= [NuFisM[state][nx][ny][nz][group]/keff for group in range(Flux.shape[-1])]
                Nu_FluxM=[NuFisL[group]*FluxL[group]*Vmesh for group in range(Flux.shape[-1])]
    
                Bet_k = BetaM[state][nx][ny][nz]
                Lamb_k = LmkM[state][nx][ny][nz]
                Nu_Flux = sum(Nu_FluxM)
    
                C0[state][nx][ny][nz] = [Bet_k[prec]*Nu_Flux/Lamb_k[prec]\
                                              for prec in range(NPRC)]#leve error numerico sin introducir lambda
                
                
                #if nx == Ix and ny == Iy and nz == Iz:
                    #print(EForm(*FluxL))
                    #print(EForm(*NuFisL))
                    #print(EForm(*Nu_FluxM))
                    #print(EForm(Nu_Flux))
                    #print(EForm(Fissions[state,nx,ny,nz]))
                    
                    #Decays = [ LmkM[state][nx][ny][nz][prc]*C0[state][nx][ny][nz][prc]\
                    #          for prc in range(NPRC)]
                    
                    #print(EForm(Precs[state,nx,ny,nz]))
                    #print(EForm(sum(Decays)))
                    
                    #print(Bet_k)
                    #print('{:.8e}'.format(Nu_FluxM[1]))
                    #print(Lamb_k)

alf=np.array(list(C0[0][22][11].values()))
lmkr=np.array(lmks)
print(EForm((lmkr*alf).sum(axis=1)[Iz]))

SE VERIFICA QUE EL CALCULO DE PRECURSORES ES CORRECTO

import matplotlib.pyplot as plt
alf=lmkr*alf
plt.plot(range(32),alf.sum(axis=1),'-')
plt.plot(range(32),Precs[0,22,11,:],'o')
plt.show()

$$ C_{i,k,t} = C_{i,k,t-1} e^{-\lambda_k \delta t} + \frac{\beta_k}{\lambda_k} \sum_g V_{mesh,i} \nu \Sigma_{f,i,g} \phi_{i,g,t-1} (1-e^{-\lambda \delta t})$$

In [101]:
def PrecCalc(C_t_1,Flux,NuFisM,Vmesh,dt):
    C_t = {}
    state = 0 #hace falta definir un estado
    C_t[state] = {}
    for nx in range(Flux.shape[1]):
        C_t[state][nx] = {}
        for ny in range(Flux.shape[2]):
            C_t[state][nx][ny] = {}
            for nz in range(Flux.shape[3]):
                
                FluxL = [Flux[state,nx,ny,nz,group] for group in range(Flux.shape[-1])]
                NuFisL= [NuFisM[state][nx][ny][nz][group] for group in range(Flux.shape[-1])]
                
                Bet_k = BetaM[state][nx][ny][nz]
                Lamb_k = LmkM[state][nx][ny][nz]
                _c_t_1 = C_t_1[state][nx][ny][nz]
                
                Nu_FluxM=[NuFisL[group]*FluxL[group]*Vmesh for group in range(Flux.shape[-1])]
                Nu_Flux = sum(Nu_FluxM)
                
                C_t[state][nx][ny][nz] = [ _c_t_1[prec]*np.exp(-Lamb_k[prec]*dt) + \
                                           Bet_k[prec]/Lamb_k[prec]*Nu_Flux*(1-np.exp(-Lamb_k[prec]*dt))\
                                            for prec in range(NPRC)]
    return C_t

In [102]:
keff

1.00173299808669

In [103]:
import re
p=re.compile('POWER\(WATTS\)\s+([0-9]\.[0-9]{5}E\+[0-9]{2})')
import os
powers=[[1e+06]]

In [ ]:
Q={}
EQUILIBRIUM=True


dt=0.001
tfinal=10

for t in np.arange(0,tfinal,dt):
    if EQUILIBRIUM:
        C_t_1 = C0.copy()
        C_t = C0.copy()
    else:
        TFlux = MeshFlux(ROOTFILE+'.cdb.meshflux',Nodosx,Nodosy,Nodosz,Ngrupos)
        C_t = PrecCalc(C_t_1,TFlux,NuFisM,Vmesh,dt)
        C_t_1 = C_t.copy()
    for group in range(Flux.shape[-1]):
        Q[group]={}
        for state in range(Flux.shape[0]):
            Q[group][state]={}
            for nx in range(Flux.shape[1]):
                Q[group][state][nx] = {}
                for ny in range(Flux.shape[2]):
                    Q[group][state][nx][ny] = {}
                    for nz in range(Flux.shape[3]):
                        _Lmk= LmkM[state][nx][ny][nz]
                        _C  = C_t[state][nx][ny][nz]
                        Q[group][state][nx][ny][nz] = chi_g[group]*sum([ _Lmk[prc]*_C[prc] for prc in range(NPRC)])
                        #de la fuente for dummies
                        #Q[group][state][nx][ny][nz] = chi_g[group]*Precs[state,nx,ny,nz]

    with open('source.dat','w') as fod:
        for group in range(NOGBIB):
            for state in range(Flux.shape[0]):
                for nz in range(Flux.shape[3]):
                    for ny in range(Flux.shape[2]):
                        for nx in range(Flux.shape[1]):
                            fod.write('{:15.7E}'.format(Q[group][state][nx][ny][nz])) 
            fod.write('\n')

    #!ejecutar_fuente.bat
    #os.system('ejecutar_fuente.bat')
    os.system('ExSource.bat '+FILENAME+' '+str(Ngrupos))
    #os.system('timeout /T 2')
    fid = open(ROOTFILE+'.cio','r')
    powers.append([pi.groups()[0] for pi in p.finditer(fid.read())])
    fid.close()
    if EQUILIBRIUM: 
        EQUILIBRIUM = False
        #break
        #print(powers)

In [ ]:
fid = open('SoutN.dat','w')
fid.write(str(powers))
fid.close()
os.system('python temp.py')
os.system('shutdown /s')

In [22]:
#Nufis1 = np.array([2.79301382E-03,3.60267647E-02])
XS0 = np.array([1.78897996E-02,2.01498996E-02,0.278627604,2.66850994E-13,0.00000000E+00,0.00000000E+00,0.471111000])
XS1 = np.array([1.78897996E-02,2.00051703E-02,0.278627604,2.66850994E-13,0.00000000E+00,0.00000000E+00,0.471111000])
XS1[1]/XS0[1]

0.992817368678105

In [23]:
XS0 = np.array([2.02931999E-03,3.27022001E-02,2.81322002E-03,3.62874009E-02,0.177189037,0.584043205,3.44858013E-14,4.83173993E-13,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.171894997,3.54918987E-02,0.00000000E+00,0.707484007])
XS1 = np.array([2.02931999E-03,3.27022001E-02,2.79301382E-03,3.60267647E-02,0.177189037,0.584043205,3.44858013E-14,4.83173993E-13,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.00000000E+00,0.171894997,3.54918987E-02,0.00000000E+00,0.707484007])
XS1[NOGBIB:2*NOGBIB]/XS0[NOGBIB:2*NOGBIB]

array([0.99281741, 0.99281745])

In [88]:
+5.374689E+16
1- (9.78897E+18)/9.84272E+18 + grp(ROOTFILE[:-1]+'.plt')[-1]/100000

0.007190888859989965

In [44]:
2.01498996E-02/1.78897996E-02 

1.1263345621825747

In [24]:
powa = reduce(add,powers)
powa=list(map(float,powa))

#value = powa.pop(1)
indx = powa.index(value)

powa = [1e+06]+powa[indx:]
plt.plot(np.r_[0.0,np.arange(0,tfinal,dt)],powa,'-o')
#plt.plot(powa,'o')
plt.show()


NameError: name 'value' is not defined

In [ ]:
powb = zip(np.r_[0.0,np.arange(0,tfinal,dt)],\
           powa)

with open('Sout2.dat','w') as fod:
    fod.write(reduce(add,map(lambda p:'{0:.2f} {1}\n'.format(p[0],p[1]),powb)))
len(powa)

In [ ]:
len(powa)

In [ ]:
powa

In [ ]:
plt.plot([powa[t+1]/powa[t] for t in range(len(powa)-1)])
plt.show()

In [ ]:
dt=0.01
tfinal=7
np.arange(0,tfinal,dt)

 NUMBER OF---COLUMNS, ROWS, PLANES, GROUPS, UPSCAT, DOWNSCAT, REGIONS, AND ZONES      44  22  32   1   0   0  29040   1999


TOTAL FIXED SOURCE   5.421056E+14 N/SEC